In [1]:
import asyncio
import json
import os
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## 1. lépés: Pydantic modellek meghatározása strukturált kimenetekhez

Ezek a modellek határozzák meg az **sémát**, amelyet az ügynökök visszaadnak. A `response_format` használata Pydantic-kal biztosítja:
- ✅ Típusbiztos adatkinyerés
- ✅ Automatikus validáció
- ✅ Nincsenek szöveges válaszokból eredő elemzési hibák
- ✅ Könnyű feltételes útvonalválasztás mezők alapján


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## 2. lépés: Hozza létre a Szállásfoglalási Eszközt

Ez az eszköz az, amit az **availability_agent** fog hívni, hogy ellenőrizze, vannak-e szabad szobák. Az `@ai_function` dekorátort használjuk arra, hogy:
- Egy Python-függvényt AI által hívható eszközzé alakítsunk
- Automatikusan JSON-sémát generáljunk az LLM számára
- Kezeljük a paraméterek érvényesítését
- Lehetővé tegyük az ügynökök automatikus meghívását

Ehhez a bemutatóhoz:
- **Stockholm, Seattle, Tokió, London, Amszterdam** → Vannak szobák ✅
- **Minden más város** → Nincsenek szobák ❌


In [3]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## 3. lépés: Feltétel-függvények meghatározása az útvonalhoz

Ezek a függvények megvizsgálják az ügynök válaszát, és eldöntik, hogy a munkafolyamat melyik útvonalát kövesse.

**Kulcsminta:**
1. Ellenőrizze, hogy az üzenet `AgentExecutorResponse`-e
2. Elemezze a strukturált kimenetet (Pydantic modell)
3. Adjon vissza `True` vagy `False` értéket az útvonalvezérléshez

A munkafolyamat ezeket a feltételeket az **éleken** értékeli ki, hogy eldöntse, melyik végrehajtót hívja meg következőként.


In [4]:
def has_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels ARE available.
    
    Returns True if the destination has hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return True  # Default to True if unexpected type

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels are NOT available.
    
    Returns True if the destination has no hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception as e:
        return False


print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)


## 4. lépés: Egyéni megjelenítő végrehajtó létrehozása

A végrehajtók olyan munkafolyamat-komponensek, amelyek átalakításokat vagy mellékhatásokat hajtanak végre. Az `@executor` dekorátort használjuk egy egyéni végrehajtó létrehozásához, amely megjeleníti a végső eredményt.

**Kulcsfogalmak:**
- `@executor(id="...")` - Egy függvényt regisztrál munkafolyamat-végrehajtóként
- `WorkflowContext[Never, str]` - Típusjelölések a bemenethez/kimenethez
- `ctx.yield_output(...)` - A munkafolyamat végső eredményét adja vissza


In [5]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created with @executor decorator")

✅ display_result executor created with @executor decorator


## 5. lépés: Környezeti változók betöltése

Konfigurálja az LLM klienst. Ez a példa az alábbiakkal működik:
- **GitHub modellek** (Ingyenes csomag GitHub tokennel)
- **Azure OpenAI**
- **OpenAI**


In [6]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")

## 6. lépés: AI-ügynökök létrehozása strukturált kimenetekkel

Három **specializált ügynököt** hozunk létre, mindegyiket egy `AgentExecutor`-ba csomagolva:

1. **availability_agent** - Ellenőrzi a szállodai elérhetőséget az eszköz segítségével
2. **alternative_agent** - Alternatív városokat javasol (ha nincs szabad szoba)
3. **booking_agent** - Foglalásra ösztönöz (ha vannak szabad szobák)

**Főbb jellemzők:**
- `tools=[hotel_booking]` - Az eszközt biztosítja az ügynök számára
- `response_format=PydanticModel` - Strukturált JSON kimenetet kényszerít
- `AgentExecutor(..., id="...")` - Ügynök becsomagolása a munkafolyamat használatához


In [7]:
# Agent 1: Check availability with tool
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## 7. lépés: A munkafolyamat felépítése feltételes élekkel

Most a `WorkflowBuilder` segítségével építjük fel a gráfot feltételes útvonalvezetéssel:

**Munkafolyamat felépítése:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙         ↘
[no_availability]  [has_availability]
        ↓              ↓
alternative_agent  booking_agent
        ↓              ↓
    display_result ←───┘
```

**Kulcsfontosságú metódusok:**
- `.set_start_executor(...)` - Beállítja a belépési pontot
- `.add_edge(from, to, condition=...)` - Feltételes él hozzáadása
- `.build()` - A munkafolyamat véglegesítése


In [8]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing:</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result
        </p>
    </div>
""")
)

## 8. lépés: Futtassuk az 1. tesztesetet - Város ELÉRHETŐSÉG NÉLKÜL (Párizs)

Teszteljük a **nincs elérhetőség** útvonalat azzal, hogy szállodákat kérünk Párizsban (amelyben nincs szoba a szimulációnk szerint).


In [9]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → alternative_agent → display_result</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run the workflow
events_paris = await workflow.run(request_paris)
outputs_paris = events_paris.get_outputs()

# Display results
if outputs_paris:
    result_paris = AlternativeResult.model_validate_json(outputs_paris[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT (Paris)</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_paris.alternative_destination}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_paris.reason}</p>
            </div>
        </div>
    """)
    )

## 9. lépés: Futtassa a 2. tesztesetet - Város ELÉRHETŐSÉGGEL (Stockholm)

Most teszteljük az **elérhetőség** útvonalat azzal, hogy szállodákat kérünk Stockholmban (ahol a szimulációnk szerint vannak szabad szobák).


In [10]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run the workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
            </div>
        </div>
    """)
    )

## Főbb tanulságok és következő lépések

### ✅ Amit megtanultál:

1. **WorkflowBuilder minta**
   - Használd a `.set_start_executor()` metódust a belépési pont meghatározásához
   - Használd a `.add_edge(from, to, condition=...)` metódust feltételes útvonalak létrehozásához
   - Hívd meg a `.build()` metódust a munkafolyamat véglegesítéséhez

2. **Feltételes útvonalak**
   - A feltétel függvények az `AgentExecutorResponse` objektumot vizsgálják
   - Strukturált kimeneteket elemezve hoznak döntéseket az útvonalakról
   - `True` értéket adnak vissza az él aktiválásához, `False` értéket az átugráshoz

3. **Eszközök integrációja**
   - Használd az `@ai_function` dekorátort Python függvények AI eszközökké alakításához
   - Az ügynökök automatikusan hívják az eszközöket, amikor szükséges
   - Az eszközök JSON-t adnak vissza, amelyet az ügynökök elemezhetnek

4. **Strukturált kimenetek**
   - Használj Pydantic modelleket típusbiztos adatkinyeréshez
   - Állítsd be a `response_format=MyModel` paramétert ügynökök létrehozásakor
   - Elemezd a válaszokat a `Model.model_validate_json()` metódussal

5. **Egyedi végrehajtók**
   - Használd az `@executor(id="...")` dekorátort munkafolyamat-komponensek létrehozásához
   - A végrehajtók átalakíthatják az adatokat vagy végezhetnek mellékhatásokat
   - Használd a `ctx.yield_output()` metódust a munkafolyamat eredményeinek előállításához

### 🚀 Valós alkalmazások:

- **Utazási foglalás**: Elérhetőség ellenőrzése, alternatívák javaslata, opciók összehasonlítása
- **Ügyfélszolgálat**: Útvonal meghatározása probléma típusa, érzelem vagy prioritás alapján
- **E-kereskedelem**: Készlet ellenőrzése, alternatívák javaslata, rendelések feldolgozása
- **Tartalom moderálás**: Útvonal meghatározása toxikussági pontszámok vagy felhasználói jelzések alapján
- **Jóváhagyási munkafolyamatok**: Útvonal meghatározása összeg, felhasználói szerepkör vagy kockázati szint alapján
- **Többlépcsős feldolgozás**: Útvonal meghatározása adatok minősége vagy teljessége alapján

### 📚 Következő lépések:

- Bonyolultabb feltételek hozzáadása (több kritérium)
- Ciklusok megvalósítása munkafolyamat-állapot kezelésével
- Al-munkafolyamatok hozzáadása újrahasznosítható komponensekhez
- Integráció valós API-kkal (szállodai foglalás, készletrendszerek)
- Hibakezelés és tartalék útvonalak hozzáadása
- Munkafolyamatok vizualizálása a beépített vizualizációs eszközökkel



---

**Felelősség kizárása**:  
Ez a dokumentum az [Co-op Translator](https://github.com/Azure/co-op-translator) AI fordítási szolgáltatás segítségével került lefordításra. Bár törekszünk a pontosságra, kérjük, vegye figyelembe, hogy az automatikus fordítások hibákat vagy pontatlanságokat tartalmazhatnak. Az eredeti dokumentum az eredeti nyelvén tekintendő hiteles forrásnak. Kritikus információk esetén javasolt professzionális emberi fordítást igénybe venni. Nem vállalunk felelősséget semmilyen félreértésért vagy téves értelmezésért, amely a fordítás használatából eredhet.
